In [1]:
#Welcome to the SQL mini project. You will carry out this project partly in
#the PHPMyAdmin interface, and partly in Jupyter via a Python connection.

#This is Tier 2 of the case study, which means that there'll be less guidance for you about how to setup
#your local SQLite connection in PART 2 of the case study. This will make the case study more challenging for you: 
#you might need to do some digging, aand revise the Working with Relational Databases in Python chapter in the previous resource.

#Otherwise, the questions in the case study are exactly the same as with Tier 1. 

#PART 1: PHPMyAdmin
#You will complete questions 1-9 below in the PHPMyAdmin interface. 
#Log in by pasting the following URL into your browser, and
#using the following Username and Password:

#URL: https://sql.springboard.com/
#Username: student
#Password: learn_sql@springboard

#The data you need is in the "country_club" database. This database
#contains 3 tables:
 #   i) the "Bookings" table,
  #  ii) the "Facilities" table, and
   # iii) the "Members" table.

#In this case study, you'll be asked a series of questions. You can
#solve them using the platform, but for the final deliverable,
#paste the code for each solution into this script, and upload it
#to your GitHub.

#Before starting with the questions, feel free to take your time,
#exploring the data, and getting acquainted with the 3 tables. */


#/* QUESTIONS 
#/* Q1: Some of the facilities charge a fee to members, but some do not.
#Write a SQL query to produce a list of the names of the facilities that do. */

#SELECT name
#FROM  `Facilities` 
#WHERE (
#membercost >0
#)



#/* Q2: How many facilities do not charge a fee to members? */

#SELECT COUNT( name ) 
#FROM  `Facilities` 
#WHERE (
#membercost =0
#)


#/* Q3: Write an SQL query to show a list of facilities that charge a fee to members,
#where the fee is less than 20% of the facility's monthly maintenance cost.
#Return the facid, facility name, member cost, and monthly maintenance of the
#facilities in question. */

#SELECT facid, name, membercost, monthlymaintenance
#FROM  `Facilities` 
#WHERE (
#membercost < monthlymaintenance * .2
#)


#/* Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
#Try writing the query without using the OR operator. */

#SELECT * 
#FROM  `Facilities` 
#WHERE facid =1
#OR facid =5
#LIMIT 0 , 30


#/* Q5: Produce a list of facilities, with each labelled as
#'cheap' or 'expensive', depending on if their monthly maintenance cost is
#more than $100. Return the name and monthly maintenance of the facilities
#in question. */

#SELECT name, monthlymaintenance, 
#CASE 
#WHEN monthlymaintenance <100
#THEN  'cheap'
#WHEN monthlymaintenance >100
#THEN  'expensive'
#END AS maintenance_cost
#FROM Facilities
#LIMIT 0 , 30



#/* Q6: You'd like to get the first and last name of the last member(s)
#who signed up. Try not to use the LIMIT clause for your solution. */

#SELECT firstname, surname
#FROM  `Members` 
#WHERE joindate = ( 
#SELECT MAX( joindate ) 
#FROM Members )


#/* Q7: Produce a list of all members who have used a tennis court.
#Include in your output the name of the court, and the name of the member
#formatted as a single column. Ensure no duplicate data, and order by
#the member name. */


#SELECT DISTINCT CONCAT(firstname,' ', surname) AS name, 
#CASE WHEN facid=0 THEN 'Tennis Court 1' 
#WHEN facid=1 THEN 'Tennis Court 2' 
#ELSE null 
#END AS tennis_courts
#FROM  `Members` AS m
#LEFT JOIN Bookings AS b 
#ON m.memid = b.memid
#WHERE facid= 0 OR facid=1
#GROUP BY m.memid
#ORDER BY name;

#/* Q8: Produce a list of bookings on the day of 2012-09-14 which
#will cost the member (or guest) more than $30. Remember that guests have
#different costs to members (the listed costs are per half-hour 'slot'), and
#the guest user's ID is always 0. Include in your output the name of the
#facility, the name of the member formatted as a single column, and the cost.
#Order by descending cost, and do not use any subqueries. */

#SELECT Facilities.name AS facility, 
#CASE WHEN (Bookings.memid>0) THEN CONCAT(firstname, ' ', surname)
#ELSE 'Guest'
#END AS Member_name,
#CASE WHEN Bookings.memid>0 THEN (Bookings.slots * Facilities.membercost)
#ELSE (Bookings.slots*Facilities.guestcost) 
#END AS cost
#FROM
#Bookings
#LEFT JOIN Facilities
#ON
#Bookings.facid = Facilities.facid
#LEFT JOIN Members
#ON
#Bookings.memid=Members.memid
#WHERE
#DATE(starttime) = '2012-09-14'
#AND
#cost > 30
#ORDER BY
#cost DESC
#;


#/* Q9: This time, produce the same result as in Q8, but using a subquery. */
#SELECT Facility, Member_name, cost
#SELECT Facility, Member_name, cost
#FROM ( 
 #   SELECT name as Facility,
  #  CASE WHEN (Bookings.memid>0)
   # 	THEN CONCAT(firstname, ' ', surname)
    #ELSE
    #	'Guest'
    #END AS Member_name,
    #CASE WHEN Bookings.memid>0
#    	THEN slots*Facilities.membercost
#    ELSE
#    END AS cost
#    FROM
#    	Bookings
#    LEFT JOIN Facilities
#    ON Bookings.facid=Facilities.facid
#    LEFT JOIN Members
#    ON Bookings.memid = Members.memid
#    WHERE
#    DATE(starttime) = '2012-09-14')
#AS sub
#WHERE cost > 30
#ORDER BY
#cost DESC
#;


In [2]:
# PART 2: SQLITE
# Export the country club data from PHPMYAdmin, and connect to a local
# SQLite instance from Juypter notebook for the following questions

#QUESTION:
#Q10: Produce a lit of facilities with a total revenue less than 1000. 
# The output of the facility name and total revenue, sorted by revenue.
# Remember that there's a different cost for guests and members!

In [ ]:
SELECT name, revenue FROM (
     SELECT Facilities.name, sum(CASE WHEN memid > 0 THEN slots*membercost
     ELSE slots*guestcost END) AS revenue
    FROM Bookings 
    INNER JOIN Facilities
    ON Facilities.facid = Bookings.facid
    GROUP BY Facilities.name
) as agg WHERE revenue < 1000
ORDER BY revenue;
 

In [3]:
import sqlite3

In [9]:
# import sqlite3
from sqlite3 import Error
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    query1 = """
       SELECT * FROM Facilities
      
        """
    cur.execute(query1)
 
    rows = cur.fetchall()
 
    for row in rows:
        print(row)


def main():
    database = "sqlite_db_pythonsqlite_db.sql"
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        print("2. Query all tasks")
        select_all_tasks(conn)
 
 
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks


DatabaseError: file is not a database

In [3]:
#Q11: Produce a report of members and who reommended them in alphabetic
# surname, firstname order 
SELECT CONCAT(m2.surname, ", ", m2.firstname) AS Members_Names, CONCAT (m1.surname, ", ", m1.firstname) AS recommenders
FROM Members AS m1
INNER JOIN Members AS m2 
ON m2.recommendedby = m1.memid
WHERE m2.recommendedby IS NOT NULL
ORDER BY
Members_Names
;

In [4]:
#Q12: Find the facilities with their usuage by member, but not guests.
SELECT Bookings.memid, CONCAT(Members.surname, ", ", Members.firstname) AS  member_name, Facilities.name AS Facility FROM Bookings 
INNER JOIN Facilities
ON Facilities.facid = Bookings.facid
INNER JOIN Members
ON Members.memid = Bookings.Memid
WHERE Bookings.memid>0
ORDER BY Bookings.memid
;


In [ ]:
#Q13: Find the facilities usage by month, but not guests
SELECT MONTH(Bookings.starttime) AS Month, Bookings.memid, Facilities.name FROM Facilities
INNER JOIN Bookings
ON Bookings.facid = Facilities.facid
WHERE
Bookings.memid > 0
GROUP BY Month
ORDER BY Month

SELECT MONTH(Bookings.starttime) AS Month, Bookings.memid, Facilities.name FROM Facilities
INNER JOIN Bookings
ON Bookings.facid = Facilities.facid
WHERE
Bookings.memid > 0
GROUP BY Month
ORDER BY Month

